[View in Colaboratory](https://colab.research.google.com/github/wonseokjung/hands_on_tf/blob/master/1_Tensorflow_eager_basic.ipynb)

In [0]:
# Tensorflow

In [0]:
import os
os.environ['DISABLE_COLAB_TF_IMPORT_HOOK'] = '1'


# tf.add( ) and tf.matmul( )


The tensorflow code below decalres five tensors names A,B,a,b, and c. 



Tensorflow can be regarded as a library of scientific calculation. 

Python code below is that calculating addition and multiplication using tensorflow eager. 


# tf.matmul( )

In [2]:
import tensorflow as tf

tf.enable_eager_execution()

A = tf.constant([[1,2],[3,4]])

B = tf.constant([[5,6],[7,8]])

C = tf.matmul(A,B)

print(A)

print(B)

print(C)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[5 6]
 [7 8]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


# tf.add( ) 



In [3]:
a = tf.constant(3)

b = tf.constant(10)

c = tf.constant(23)

d = tf.add(a,b,c)

print(d)

tf.Tensor(13, shape=(), dtype=int32)


# Automatic Differentiation Mechanism for differentiation


In [6]:
import tensorflow as tf

tf.enable_eager_execution()

x = tf.get_variable('x', shape = [1])

print(x)

x = tf.get_variable('x', shape=[1], initializer = tf.constant_initializer(3.))

print(x)

with tf.GradientTape() as tape : 
  y= tf.square(x)
  
y_grad = tape.gradient(y, x)

print([y.numpy(), y_grad.numpy()])

<tf.Variable 'x:0' shape=(1,) dtype=float32, numpy=array([0.52280605], dtype=float32)>
<tf.Variable 'x:0' shape=(1,) dtype=float32, numpy=array([3.], dtype=float32)>
[array([9.], dtype=float32), array([6.], dtype=float32)]


x is variable and initialized to 3. 

x is declared by tf.get_variable()

Variable have shape, dtype attributes, and initializer. 

We can set the Initializer by Initializer parameter. 

Here we use tf.constant_initializer(3.0) to initialize the variable x to 3.0 with a float 32 dtype.

tf.GradientTape( ) is a recorder of automatic differentation. It records all variable and steps of calculation automatically. 

For the example, the variable x and the calculation step y=tf.square(x) are recoreded automatically. 

So the derivate of the tensor y with respect to x can be obtained through y_grad = tape.gradient(y, x )




In machine learning, calcuating the derivative of a multivariable function, a vector or a matrix is common case which is a piece cake for Tensorflow. 

The following code show how to utilize tf.GradientTape ( ) to differentiate 

$$L(w,b) = || Xw + b - y ||^2$$

with respect to $w$ and $b$ at $w$ and $b$ at $w = (1,2)^{T}, b = 1$



In [16]:
X = tf.constant([[1.,2.], [3.,4.]])

y = tf.constant([[1.],[2.]])

w = tf.get_variable('w', shape=[2,1], initializer = tf.constant_initializer([[1.],[2.]]))

b = tf.get_variable('b', shape = [1], initializer = tf.constant_initializer([1.]))

with tf.GradientTape() as tape: 
  L = 0.5*tf.reduce_sum(tf.square(tf.matmul(X,w) + b - y))
  
w_grad, b_grad = tape.gradient(L, [w,b])

print([L.numpy(), w_grad.numpy(), b_grad.numpy()])
                    
                    

[62.5, array([[35.],
       [50.]], dtype=float32), array([15.], dtype=float32)]


😊


tf.square ( ) : squares elements in the input tensor

tf.reduce_sum( ) : outputs the sum of all elements in the input tnesor with a null shape 


<img src="https://www.dropbox.com/s/2vrlyxgfoktfhg9/Screenshot%202018-09-27%2011.47.50.png?raw=1">




# Example : Linear Regression



The house price of a city  between 2013~ 2017. 


<img src="https://www.dropbox.com/s/h3ldtci5371spjc/Screenshot%202018-09-27%2011.50.20.png?raw=1">

1. We wamnt to do linear regression given data. 

ex ) using the linear model $ y = ax + b$ , $a$ and $b$ are unkonwon parameter 



In [22]:
import numpy as np 

X_raw = np.array([2013,2014,2015,2016,2017], dtype = np.float32)

y_raw = np.array([12000,14000,15000,16500,17500],dtype = np.float32)


X = (X_raw - X_raw.min()) / (X_raw.max()- X_raw.min()) 

y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())


print(X)

print(y)


[0.   0.25 0.5  0.75 1.  ]
[0.         0.36363637 0.54545456 0.8181818  1.        ]


We use gradient descent in order to evaluate two parameters a and b. 

To search local minima of a multivariable function $f(x)$, we use gradient descent following steps present below : 

1. Initialize the augument to $x_0$ and have k = 0 

2. Iterate the following steps repeatedly till the convergence criteria is met : 
 
      2.1 Find the gradient of the function $f(x)$ with respect to the parameter $\nabla f(x_k)$
      2.2 Update the parameter x_{k_1} = x_k - \gamma \nabla f(x_k)$ where $\gamma $ is the learning rate 
      
      2.3 $k$ $\leftarrow$ $k+1$

# TensorFlow Eager mode have similar operations as Numpy 



In [0]:
X = tf.constant(X)

y = tf.constant(y)

a = tf.get_variable('a', dtype = tf.float32, shape = [] , initializer = tf.zeros_initializer)

b = tf.get_variable('b', dtype = tf.float32, shape = [] , initializer = tf.zeros_initializer)


variables = [a,b]

In [32]:
num_epoch = 10000 

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-3)

for e in range(num_epoch): 
  with tf.GradientTape() as tape: 
    y_pred = a* X+b
    loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y ))
    
  grads = tape.gradient(loss,variables)
  optimizer.apply_gradients(grads_and_vars= zip(grads, variables))
 
           
  


0 tf.Tensor(0.004545458, shape=(), dtype=float32)
1 tf.Tensor(0.004545458, shape=(), dtype=float32)
2 tf.Tensor(0.004545458, shape=(), dtype=float32)
3 tf.Tensor(0.004545458, shape=(), dtype=float32)
4 tf.Tensor(0.004545458, shape=(), dtype=float32)
5 tf.Tensor(0.004545458, shape=(), dtype=float32)
6 tf.Tensor(0.004545458, shape=(), dtype=float32)
7 tf.Tensor(0.004545458, shape=(), dtype=float32)
8 tf.Tensor(0.004545458, shape=(), dtype=float32)
9 tf.Tensor(0.004545458, shape=(), dtype=float32)
10 tf.Tensor(0.004545458, shape=(), dtype=float32)
11 tf.Tensor(0.004545458, shape=(), dtype=float32)
12 tf.Tensor(0.004545458, shape=(), dtype=float32)
13 tf.Tensor(0.004545458, shape=(), dtype=float32)
14 tf.Tensor(0.004545458, shape=(), dtype=float32)
15 tf.Tensor(0.004545458, shape=(), dtype=float32)
16 tf.Tensor(0.004545458, shape=(), dtype=float32)
17 tf.Tensor(0.004545458, shape=(), dtype=float32)
18 tf.Tensor(0.004545458, shape=(), dtype=float32)
19 tf.Tensor(0.004545458, shape=(), dtype